In [2]:
import findspark
findspark.find()
findspark.init(edit_profile=True)
import pyspark
sc = pyspark.SparkContext()

In [3]:
import re
import copy

In [4]:
rawData = sc.textFile('pg100.txt')

In [5]:
wlist = rawData.flatMap(lambda x: x.split(' ')).filter(lambda x: not x.isdigit()).filter(lambda x: not x.isupper()).filter(lambda x: x != '')

In [141]:
# withIndex = wlist.zipWithIndex().map(lambda (k,v): (v,k)) 
# withIndex.map(lambda x: (x[1], withIndex.lookup(x[0] + 1), withIndex.lookup(x[0] + 2))).take(10)

In [16]:
wlist_2 = wlist.collect()
word_seq = []
for i in xrange(len(wlist_2)-2):
    word_seq.append(((wlist_2[i], wlist_2[i+1]),wlist_2[i+2]))

In [89]:
def getWordCount(wlist):
    result = {}
    for i in wlist:
        try:
            result[i] = result[i] + 1
        except: 
            result[i] = 1
    result = sorted(result.items(), key=lambda x: x[1], reverse = True)
    return result

In [92]:
resultRDD = sc.parallelize(word_seq).groupByKey().mapValues(lambda x: list(x)).mapValues(getWordCount).cache()

In [98]:
resultRDD.take(5)

[((u'so', u'was'),
  [(u"I'll", 1), (u'mine,', 1), (u'he', 1), (u'this.', 1), (u'he;', 1)]),
 ((u"worship's", u'friend,'), [(u'and', 1)]),
 ((u'Faith,', u'some'), [(u'certain', 1)]),
 ((u'four.', u'Poins.'), [(u'Ay,', 1)]),
 ((u'honour', u'he'), [(u"requir'd;", 1)])]

In [125]:
tmp = resultRDD.keys().takeSample(True, 1)[0]
print tmp
print resultRDD.map(lambda x: x).lookup((1, 1))
# resultRDD.keys().takeSample(True, 1)[0]
# resultRDD.keys().take(1)

(u'hath', u'most')
[]


In [135]:
def generateLine(word_count):
    init_key = resultRDD.keys().takeSample(True, 1)[0] 
    phrase = init_key[0] + ' ' + init_key[1]
    for i in xrange(word_count - 2):
        try:
            new_word = resultRDD.map(lambda x: x).lookup(init_key)[0][0][0]
        except: 
            init_key = resultRDD.keys().takeSample(True, 1)[0]
            new_word = resultRDD.map(lambda x: x).lookup(init_key)[0][0][0]
        
        phrase = phrase + ' ' + new_word
        init_key = (init_key[1], new_word)
    
    return phrase
    
def multipleLines(line_count, word_count):
    lines = ''
    for i in xrange(line_count):
        lines = lines + '\n' + generateLine(word_count)
    return lines

In [136]:
Shakespeare_verse = multipleLines(10, 20)

In [140]:
print Shakespeare_verse



or wisdom, To let him be brought to light, Whose ugly and unnatural That such a one as a man
thunders and lightens] What stir is this that you have a good conscience. The very thought of this agreement for
be angry? Well, well, see the writing. do beseech you, sir, to my lord the Duke of York and Somerset
have kings that privates have not seen the day of marriage, and God defend the right! And so, good night.
boy Love is a good conscience. The very thought of this agreement for keeping the Project Gutenberg Literary Archive Foundation
have to-night dispatch'd sixteen businesses, a month's length apiece; by an by a piece of work that will not be
men's hopes; And, like a man of his own hand did stay. Romeo, that did love you bear a letter
that Pompey gives him, else is his name? His name, pray you. I'll not be a man. But art thou
mighty rotting Together have one eye and heart a league of amity, And will not be a man. But art
our hope, sir, that you have a good conscience. The very thoug